In [43]:
# Import the required libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')

# Reading the csv file
messages = pd.read_csv('./Data/SpamData',sep='\t',
                      names = ['label','message'])

# Initialize stemmer
lemmatizer = WordNetLemmatizer()

# Data cleaning and preprocessing
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['message'][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

# Creating TF-IDF vectorizer model 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(corpus).toarray()

# Creaing dummies for label
y = pd.get_dummies(messages['label'])
y = y.iloc[:,1].values.ravel()

# Handle imbalanced data
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_res,y_res = smote.fit_resample(X,y)

# Spliting data into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_res,y_res,test_size=0.2,random_state=0)

# Training the model using Naive Bayes calsssifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train,y_train)

# Prediting the trained model 
y_pred = clf.predict(X_test)

# Calculating the prediction accuracy
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy = accuracy_score(y_test,y_pred)
conf_mat = confusion_matrix(y_test,y_pred)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ciya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [44]:
conf_mat

array([[917,  25],
       [  6, 982]], dtype=int64)

In [45]:
accuracy

0.9839378238341969